In [ ]:
%pip install spyndex

In [1]:
#papermill_description=imports

import json
import os
import numpy as np
import logging
import sys
from io import StringIO

import geopandas as gpd

from gis_utils.stac import save_metadata_sidecar
from gis_utils.visualisation import get_geotiff_statistics, colour_geotiff_and_save_cog
from gis_utils.dataframe import get_bbox_from_geodf
from gis_utils.colormap import get_colormap, display_colormap_as_html

import rasterio
import requests
import rioxarray
import xarray as xr

import rasterio
import rasterio.mask  # added for masking/cliping rasters
from rasterio.windows import from_bounds
from rasterio.plot import reshape_as_raster
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.io import MemoryFile
from rasterio.enums import Resampling
from rasterio.io import MemoryFile

from rio_cogeo.cogeo import cog_translate
from rio_cogeo.profiles import cog_profiles

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize

# Set environment variable for AWS public datasets
os.environ['AWS_NO_SIGN_REQUEST'] = 'YES'


In [8]:
output_tiff_directory = '/home/jenna/workspace/sensand/repos/data-notebooks/notebooks/sandbox/data/cmi-planet/processed'


toi1_tiff = '/home/jenna/workspace/sensand/repos/data-notebooks/notebooks/sandbox/data/cmi-planet/TOI1/toi1_merged.tif'
toi2_tiff = '/home/jenna/workspace/sensand/repos/data-notebooks/notebooks/sandbox/data/cmi-planet/TOI2/toi2_merged.tif'
toi3_tiff = '/home/jenna/workspace/sensand/repos/data-notebooks/notebooks/sandbox/data/cmi-planet/TOI3/toi3_merged.tif'
toi4_tiff = '/home/jenna/workspace/sensand/repos/data-notebooks/notebooks/sandbox/data/cmi-planet/TOI4/toi4_merged.tif'
toi5_tiff = '/home/jenna/workspace/sensand/repos/data-notebooks/notebooks/sandbox/data/cmi-planet/TOI5/toi5_merged.tif'

toi1_evi_tiff = os.path.join(output_tiff_directory, 'toi1_evi.tif')
toi2_evi_tiff = os.path.join(output_tiff_directory, 'toi2_evi.tif')
toi3_evi_tiff = os.path.join(output_tiff_directory, 'toi3_evi.tif')
toi4_evi_tiff = os.path.join(output_tiff_directory, 'toi4_evi.tif')
toi5_evi_tiff = os.path.join(output_tiff_directory, 'toi5_evi.tif')

toi1_evi_coloured_tiff = os.path.join(output_tiff_directory, 'toi1_evi_coloured.tif')
toi2_evi_coloured_tiff = os.path.join(output_tiff_directory, 'toi2_evi_coloured.tif')
toi3_evi_coloured_tiff = os.path.join(output_tiff_directory, 'toi3_evi_coloured.tif')
toi4_evi_coloured_tiff = os.path.join(output_tiff_directory, 'toi4_evi_coloured.tif')
toi5_evi_coloured_tiff = os.path.join(output_tiff_directory, 'toi5_evi_coloured.tif')


input_list = [toi1_tiff, toi2_tiff, toi3_tiff, toi4_tiff, toi5_tiff]
output_evi_list = [toi1_evi_tiff, toi2_evi_tiff, toi3_evi_tiff, toi4_evi_tiff, toi5_evi_tiff]
output_evi_coloured_list = [toi1_evi_coloured_tiff, toi2_evi_coloured_tiff, toi3_evi_coloured_tiff, toi4_evi_coloured_tiff, toi5_evi_coloured_tiff]


In [9]:
for input, evi, evi_colour in zip(input_list, output_evi_list, output_evi_coloured_list):

    with rasterio.open(input) as src:
        transform = src.window_transform(window)
        raster_crs = src.crs

In [10]:
#papermill_description=processing_stac
try:
    print(f"Opening DEM asset from: {slga_cog_source}")
    data, metadata = None, {}


    with rasterio.open(slga_cog_source) as src:
        # Nodata value here is being set to 0. This works for DEM but is not OK for indices.
        data, out_transform = rasterio.mask.mask(src, coords, crop=True, filled=False, all_touched=True) #try filled boolean instead of setting nodata

        # Extract required metadata or other information from src
        metadata = src.meta.copy()

        # Update metadata with the new size and transform
        metadata.update({
            'height': data.shape[1],
            'width': data.shape[2],
            'transform': out_transform
        })
        
        
        output_directory = os.path.dirname(output_tiff_filename)
        os.makedirs(output_directory, exist_ok=True)

        if resample is True:
            upscale_factor = 2
            print(f"Resampling data to upscale factor: {upscale_factor}")

             # Calculate new transform to adjust the resolution (pixel size)
            new_transform = rasterio.Affine(
                metadata['transform'].a / upscale_factor,  # Divide pixel size by upscale factor
                metadata['transform'].b,
                metadata['transform'].c,
                metadata['transform'].d,
                metadata['transform'].e / upscale_factor,  # Divide pixel size by upscale factor
                metadata['transform'].f
            )

            # Update metadata with the new size and transform
            metadata.update({
                'height': int(data.shape[1]*upscale_factor),
                'width': int(data.shape[2] * upscale_factor),
                'transform': new_transform
            })


            with MemoryFile() as memfile:
                with memfile.open(**metadata) as dataset:
                    dataset.write(data)
                    data = dataset.read(
                        out_shape=(
                            dataset.count,
                            int(dataset.height * upscale_factor),
                            int(dataset.width * upscale_factor)
                        ),
                        resampling=Resampling.bilinear
                    )

                    

                    with rasterio.open(output_tiff_filename, 'w', **metadata) as dst:
                        dst.write(data)
                        print(f"Written masked data to {output_tiff_filename}")

                    # Optionally, log the size of the written file
                    output_file_size = os.path.getsize(output_tiff_filename)
                    print(f"Output mask file size: {output_file_size} bytes")


        else:
            print(f"Writing to file: {output_tiff_filename}")
            with rasterio.open(output_tiff_filename, 'w', **metadata) as dst:
                dst.write(data)
                print(f"Written masked data to {output_tiff_filename}")

            # Optionally, log the size of the written file
            output_file_size = os.path.getsize(output_tiff_filename)
            print(f"Output mask file size: {output_file_size} bytes")

except Exception as e:
    logging.error(f"Error processing asset: {e}", exc_info=True)

Opening DEM asset from: https://esoil.io/TERNLandscapes/Public/Products/TERN/SLGA/CLY/CLY_000_005_EV_N_P_AU_TRN_N_20210902.tif
Writing to file: /workspace/notebooks/sandbox/data/slga_testing/SLGA_Clay_0-5cm_test_masked.tiff
Written masked data to /workspace/notebooks/sandbox/data/slga_testing/SLGA_Clay_0-5cm_test_masked.tiff
Output mask file size: 65728 bytes


In [12]:
#papermill_description=processing_cog

with rasterio.open(output_tiff_filename) as mew:
    meta = mew.meta.copy()
    dst_crs = rasterio.crs.CRS.from_epsg(4326)
    transform, width, height = calculate_default_transform(
        mew.crs, dst_crs, mew.width, mew.height, *mew.bounds
    )

    meta.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    tif_data = mew.read(1, masked=True).astype('float32') #setting masked=True here tells rasterio to use masking information if present, but we need to add the mask itself first.
    mew_formatted = tif_data.filled(np.nan)

    cmap = cm.get_cmap(colormap) #can also use 'terrain' cmap to keep this the same as the preview image from above.
    na = mew_formatted[~np.isnan(mew_formatted)]

    min_value = min(na)
    max_value = max(na)

    norm = Normalize(vmin=min_value, vmax=max_value)

    coloured_data = (cmap(norm(mew_formatted))[:, :, :3] * 255).astype(np.uint8)

    meta.update({"count":3})


    with rasterio.open(output_coloured_tiff_filename, 'w', **meta) as dst:
        reshape = reshape_as_raster(coloured_data)
        dst.write(reshape)

try:
    dst_profile = cog_profiles.get('deflate')
    with MemoryFile() as mem_dst:
        cog_translate(
            output_coloured_tiff_filename,
            output_cog_tiff_filename,
            config=dst_profile,
            in_memory=True,
            dtype="uint8",
            add_mask=False,
            nodata=0,
            dst_kwargs=dst_profile
        )
    
    save_metadata_sidecar(output_cog_tiff_filename, asset_metadata)    
except Exception as e:
    logging.error(f"Error processing asset: {e}")


/tmp/ipykernel_36404/2783041225.py:20: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap(colormap) #can also use 'terrain' cmap to keep this the same as the preview image from above.
Reading input: /workspace/notebooks/sandbox/data/slga_testing/SLGA_Clay_0-5cm_test_coloured.tiff

Updating dataset tags...
Writing output to: /workspace/notebooks/sandbox/data/slga_testing/SLGA_Clay_0-5cm_test_cog.public.tiff


Metadata saved to /workspace/notebooks/sandbox/data/slga_testing/SLGA_Clay_0-5cm_test_cog.public.tiff.meta.json


In [22]:
#papermill_description=coloring_cog_and_processing_colormap

#set the default number of colours that will appear in the legend:
color_count=12

#if there are less than the color_count number of unique values in the raster, then set the color_count to the number of unique values.
array = np.array(tif_data)
values = np.unique(array)

if len(values) < color_count:
    color_count = len(values)-1

# generate the colormap for the legend
colormap_legend = get_colormap(colormap, [raster_stats['min'], raster_stats['mean'],  raster_stats['max']], color_count, 3)

display_colormap_as_html(colormap_legend)

# add colormap to metadata
asset_metadata['legend'] = {
    'colormap': {
        'type': 'discrete',
        'colors': colormap_legend
    },
    'unit': unit
}

asset_metadata['properties']['overlayType'] = 'SLGA'

save_metadata_sidecar(output_cog_tiff_filename, asset_metadata)

Metadata saved to /workspace/notebooks/sandbox/data/slga_testing/SLGA_Clay_0-5cm_test_cog.public.tiff.meta.json
